# Projeto de desenvolvendo um modelo de regressão para previsão do preço de fechamento das ações

### Descrição do problema de regressão

No mercado financeiro, escolha do momento de realização das transações tem grande impacto a otimização dos resultados. Essa pode resultar em grandes rentabilidades em ciclos de alta, ou grandes perdas em ciclos de baixa. 

As metodologias de análise mais eficazes para apoiar os investidores nessa escolha demandam conhecimento e acesso a informações que, frequentemente, não são de domínio dos investidores. 

Pelo conhecido interesse dos investidores pelas ações WEGE3, e o fácil acesso ao seu histórico de cotações, este projeto teve como objetivo utilizá-lo para desenvolver um modelo de regressão. Este permitirá a predição do preço de fechamento das ações com um dia de antecedência, auxiliando investidores a mitigar riscos e maximizar retornos.

### Descrição dos dados











































































Os dados do histórico de cotações foram obtidos através da biblioteca yfinance, da linguagem Python. Estes foram disponibilizados no formato de um dataframe pandas, com seis colunas e as datas como índice. 

A coluna “Open” armazena os preços de abertura, a coluna “Close” os preços nominais de fechamento e a coluna “Adj Close” os preços de fechamento ajustado. As colunas “High” e “Low” armazenam, respectivamente, os preços mais altos e mais baixos do dia. Por fim, a coluna “Volume” armazena a quantidade de transações realizadas no dia.

## Parte 1

- Análise Exploratória

- Tratamento de dados

- Divisão da base de dados em treino e teste

Instalando e importando bibliotecas

In [1]:
# ! pip install --upgrade pip
# ! pip install pandas
# ! pip install yfinance
# ! pip install ta
# ! pip install -U scikit-learn
# ! pip install plotly
# ! pip install xgboost
# ! pip install setuptools
# ! pip install tensorflow

# ! pip install nbformat==4.2.0
# ! pip install --upgrade nbformat 
# # OBS.: REINICIAR O KERNEL DO JUPYTER APÓS A ATUALIZAÇÃO DO nbformat

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import ta
from sklearn.model_selection import train_test_split

# desabilita os warnings
import warnings
warnings.filterwarnings('ignore')

Importando Dados

In [3]:
ticker = "WEGE3"
df_original = yf.download(tickers=f"{ticker}.SA", start="2010-01-01")
df = df_original

[*********************100%%**********************]  1 of 1 completed


Análise Exploratória

In [4]:
# Verificando os tipos de dados e a ocorrência de dados faltantes
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3644 entries, 2010-01-04 to 2024-09-03
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3644 non-null   float64
 1   High       3644 non-null   float64
 2   Low        3644 non-null   float64
 3   Close      3644 non-null   float64
 4   Adj Close  3644 non-null   float64
 5   Volume     3644 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 199.3 KB


In [5]:
# Verificando estatisticas descritivas
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,3644.000000,3644.000000,3644.000000,3644.000000,3644.000000,3.644000e+03
mean,14.904974,15.128256,14.691937,14.903376,13.993706,5.765964e+06
std,14.070723,14.276702,13.870464,14.072247,13.867848,5.261399e+06
min,2.218934,2.332840,2.146449,2.218934,1.723242,0.000000e+00
25%,4.163831,4.211168,4.118343,4.160133,3.393147,2.895347e+06
50%,7.257692,7.351923,7.180769,7.261538,6.407748,4.814260e+06
75%,30.532501,30.945000,29.982500,30.432500,29.191412,7.271045e+06
max,54.500000,54.779999,53.770000,54.200001,54.200001,1.360207e+08


In [6]:
for coluna in df.columns:
    print(f"Mediana da coluna {coluna}: {df[coluna].median()}")

Mediana da coluna Open: 7.257692098617554
Mediana da coluna High: 7.351922988891602
Mediana da coluna Low: 7.180768966674805
Mediana da coluna Close: 7.261538028717041
Mediana da coluna Adj Close: 6.407748222351074
Mediana da coluna Volume: 4814260.0


In [7]:
px.histogram(data_frame=df,
             x= df.index,
             y = df.columns,
             title="Histogramas",
             width = 600
             )

Tratando dados

In [8]:
# Substutindo valores faltantes da coluna Volume pelo último valor válido
df['Volume'] = df['Volume'].replace(0, np.nan).fillna(method='ffill')
px.line(df['Volume'])

In [9]:
# Substutindo valores inconsistentes da coluna Volume pelo último valor válido
df['Volume'] = np.where(df['Volume'] > 50_000_000, np.nan, df['Volume'])
df['Volume'] = df['Volume'].fillna(method='ffill')
px.line(df['Volume'])

Ajustando as colunas para trabalhar somente com valores ajustados

In [10]:
df['Open'] = df['Open'] - (df['Close'] - df['Adj Close'])
df['High'] = df['High'] - (df['Close'] - df['Adj Close'])
df['Low'] = df['Low'] - (df['Close'] - df['Adj Close'])
df['Close'] = df['Adj Close']

df = df.drop(['Adj Close'], axis=1)

Criando coluna target com os retornos percentuais dos dias seguintes

In [11]:
df['Target'] = df['Close'].shift(-1)
df = df.dropna()

Criando colunas com métricas adicionais  

In [12]:
# # Criado coluna com os valores de retorno percentual diário
df['Return'] = df['Close'] - df['Open']
df['Target_Return'] = df['Return'].shift(-1)
df['Return_pct'] = (df['Close'] - df['Open'])/df['Open']
df['Target_Return_pct'] = df['Return_pct'].shift(-1)

# # Médias móveis exponenciais dos retornos 
# # nos 10 dias de negociação anteriores
df["EMA_10_Ret"] = df['Return'].ewm(span=10).mean()

# # Médias móveis aritméticas dos retornos 
# # nos 45 dias de negociação anteriores
df["MA_45_Ret"] = df['Return'].rolling(45).mean()

# # Desvio Padrão móvel dos fechamentos 
df["STD_45_Ret"] = df['Return'].rolling(45).std()

# # RSI dos retornos nos 10 dias de 
# # negociação anteriores
df['RSI_10_Ret'] = ta.momentum.RSIIndicator(close=df["Return"], window=10).rsi()/100

# Médias móveis exponenciais dos retornos 
# nos 10 dias de negociação anteriores
df["EMA_10"] = df['Close'].ewm(span=10).mean()
df["EMA_45"] = df['Close'].ewm(span=45).mean()

# Médias móveis aritméticas dos retornos 
# nos 45 dias de negociação anteriores
df["MA_10"] = df['Close'].rolling(10).mean()
df["MA_45"] = df['Close'].rolling(45).mean()

# Desvio Padrão móvel dos fechamentos 
df["STD_10"] = df['Close'].rolling(10).std()
df["STD_45"] = df['Close'].rolling(45).std()

# RSI dos retornos nos 10 dias de 
# negociação anteriores
df['RSI_10'] = ta.momentum.RSIIndicator(close=df["Close"], window=10).rsi()/100
df['RSI_45'] = ta.momentum.RSIIndicator(close=df["Close"], window=45).rsi()/100

df = df.dropna()

df['High-Low'] = df['High'] - df['Low']
df['High-Close'] = df['High'] - df['Close']
df['Close-Low'] = df['Close'] - df['Low']

df['(High-Low)/Close'] = (df['High'] - df['Low']) / df['Close']
df['(High-Close)/Close'] = (df['High'] - df['Close']) / df['Close']
df['(Close-Low)/Close'] = (df['Close'] - df['Low']) / df['Close']

df = df.drop(['Open', 'High', 'Low'], axis=1)

Análise de correlação

In [13]:
px.imshow(
    df.corr(method='pearson').round(2),
    text_auto=True,
    title="Correlação de Pearson",
    color_continuous_scale="Cividis", 
    width = 1100,
    height= 1000,
)

In [14]:
px.imshow(
    df.corr(method='spearman').round(2),
    text_auto=True,
    title="Correlação de Spearman",
    color_continuous_scale="Cividis", 
    width = 1100,
    height= 1000,
)

Selecionando features e target

In [15]:
df = df[['MA_45', 'Target']]

Otimizando tipos de dados

In [16]:
df[df.columns] = df[df.columns].apply(pd.to_numeric, downcast='float')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3598 entries, 2010-03-10 to 2024-08-30
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   MA_45   3598 non-null   float32
 1   Target  3598 non-null   float32
dtypes: float32(2)
memory usage: 56.2 KB


Divisão da base de dados em conjutos de treino e teste

In [17]:
df['ano'] = df.index.year

X_train, X_test, y_train, y_test = train_test_split(
    df.drop('Target', axis=1), 
    df['Target'], 
    test_size=0.3, 
    random_state=42,
    stratify=df['ano'])

df = df.drop('ano', axis=1)
X_train = X_train.drop('ano', axis=1)
X_test = X_test.drop('ano', axis=1)

## Parte 2

- Desenvolvimento de modelos.

- Teste de desempenho de modelos.

- Seleção do modelo.

- Avaliação do modelo.

Importando bibliotecas

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

Preparando df de métricas de avaliação dos modelos

In [19]:
df_modelos = pd.DataFrame(columns=[
    'Algoritmo', 
    'Parametros', 
    'R2_treino', 
    'R2_teste', 
    'MAE_treino', 
    'MAE_teste',
    'RMSE_treino', 
    'RMSE_teste', 
    ])

Instanciando, treinando e avaliando modelos

In [20]:
modelos = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
    'MLP': MLPRegressor(),
    }

for nome_modelo, modelo in modelos.items():

    # Definindo conjuto de hiperparametros para cada modelo 
    match nome_modelo:

        case 'Linear Regression':

            param_distributions = {'fit_intercept' : [True, False]}

        case 'Decision Tree':

            param_distributions = {
            "criterion" : ['poisson', 
                           'absolute_error', 
                           'squared_error', 
                           'friedman_mse'],
            "splitter" : ['best', 'random'],
            "max_depth" : [i for i in range(1, 3)],
            "min_samples_leaf" : [i for i in range(2, 11)],
            "min_samples_split" : [i for i in range(2, 11)],
            }

        case 'Random Forest':

            param_distributions = {
                "n_estimators" : [i for i in range(1, 3)],
                "max_depth" : [i for i in range(1, 3)],
                "max_features" : [i for i in range(1, 3)],
                "bootstrap" : [True, False],
            }

        case 'XGBoost':

            param_distributions = {
                "n_estimators" : [100, 200, 300],
                "learning_rate" : [0.1, 0.01, 0.001],
                "max_depth" : [3, 5, 7],
                "subsaples" : [0.8, 1.0],
                "colsamples_bytree" : [0.8, 1.0],
                "objective" : ['reg:squarederror'],
            }

        case 'MLP': 

            param_distributions = {
                'activation' : ['identity', 
                                'logistic', 
                                'tanh', 
                                'relu'], 
                'solver' : ['lbfgs', 'sgd', 'adam'],
                'alpha' : [ _ for _ in [0.1, 
                                      0.01, 
                                      0.001, 
                                      0.0001]],
                'learning_rate' : ['constant', 
                                   'invscaling', 
                                   'adaptive'],
                'learning_rate_init' : [ _ for _ in [0.1, 
                                      0.01, 
                                      0.001, 
                                      0.0001]],
                'power_t' : [_ for _ in list(np.arange(0.1, 0.9, 0.1))],
                'max_iter' : [_ for _ in range(50, 300, 50)],
                'shuffle' : [True, False],
                'random_state' : [42],
                'warm_start' : [True, False],
                'max_fun' : [_ for _ in range(2, 10, 1)] # defaut = 1500
            }

        # case 'LSTM':

        case _:

            pass

    # Instanciando o RandomizedSearchCV
    random_search = RandomizedSearchCV(
        modelo,
        param_distributions = param_distributions,
        n_iter=20,
        cv  =  10,
        scoring =  'neg_root_mean_squared_error',
        # verbose = 5,
        random_state = 42,
    )

    # Ajustando RandomizedSearchCV
    random_search.fit(X_train, y_train)

    # Estimativas de treino e teste
    y_pred_train = random_search.predict(X_train)
    y_pred = random_search.predict(X_test)

    modelo = pd.DataFrame({
        'Algoritmo' : nome_modelo, 
        'Parametros' : random_search.best_params_, 
        'R2_treino' : np.sqrt(r2(y_train, y_pred_train)), 
        'R2_teste' : np.sqrt(r2(y_test, y_pred)), 
        'MAE_treino' : np.sqrt(mae(y_train, y_pred_train)), 
        'MAE_teste' : np.sqrt(mae(y_test, y_pred)),
        'RMSE_treino' : np.sqrt(mse(y_train, y_pred_train)), 
        'RMSE_teste' : np.sqrt(mse(y_test, y_pred)), 
    })

    df_modelos = pd.concat([df_modelos,modelo],ignore_index=True)

In [21]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Instanciando o modelo
modelo_lstm = Sequential([
    LSTM(30, input_shape=[None,1], return_sequences=True),
    LSTM(30, return_sequences=True),
    LSTM(30),
    Dense(1)
    ])

# Compilando o modelo
modelo_lstm.compile(loss='mean_squared_error', optimizer='adam')

# Treinando o modelo (com histórico das épocas)
history = modelo_lstm.fit(X_train, y_train, epochs = 50)

# Avaliando o modelo
y_pred = modelo_lstm.predict(X_test)
y_pred_train = modelo_lstm.predict(X_train)

modelo = pd.DataFrame({
    'Algoritmo' : 'LSTM', 
    'Parametros' : 'units = 30 / epochs = 50 / optimizer = adam', 
    'R2_treino' : np.sqrt(r2(y_train, y_pred_train)), 
    'R2_teste' : np.sqrt(r2(y_test, y_pred)), 
    'MAE_treino' : np.sqrt(mae(y_train, y_pred_train)), 
    'MAE_teste' : np.sqrt(mae(y_test, y_pred)),
    'RMSE_treino' : np.sqrt(mse(y_train, y_pred_train)), 
    'RMSE_teste' : np.sqrt(mse(y_test, y_pred))},
    index=[0])

df_modelos = pd.concat([df_modelos,modelo],ignore_index=True)

Epoch 1/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 397.9492
Epoch 2/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 233.8676
Epoch 3/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - loss: 189.8863
Epoch 4/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/step - loss: 154.2356
Epoch 5/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 120.1430
Epoch 6/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 857us/step - loss: 105.5568
Epoch 7/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 92.4669
Epoch 8/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step - loss: 70.6516
Epoch 9/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 64.9986
Epoch 10/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 920us/step - loss: 52.8014
Epoch 11/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 47.1343
Epoch 12/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - loss: 37.5806
Epoch 13/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - loss: 35.5564
Epoch 14/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 25.8274
Epoch 15/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

Comparando desempenho dos modelos

In [22]:
df_modelos = df_modelos.groupby('Algoritmo').agg({'Parametros': list,
                                                  'R2_treino': 'mean',
                                                  'R2_teste': 'mean',
                                                  'MAE_treino': 'mean',
                                                  'MAE_teste': 'mean',
                                                  'RMSE_treino': 'mean',
                                                  'RMSE_teste': 'mean'})

df_modelos = df_modelos.sort_values(by='RMSE_teste')
df_modelos

,Parametros,R2_treino,R2_teste,MAE_treino,MAE_teste,RMSE_treino,RMSE_teste
Algoritmo,,,,,,,
XGBoost,"[0.8, reg:squarederror, 100, 3, 0.1, 0.8]",0.994600,0.994699,0.871403,0.885092,1.437571,1.432008
MLP,"[True, adam, True, 42, 0.30000000000000004, 25...",0.992600,0.992909,0.955114,0.954082,1.682022,1.655356
Linear Regression,[True],0.992601,0.992909,0.956495,0.955590,1.681928,1.655369
LSTM,[units = 30 / epochs = 50 / optimizer = adam],0.989938,0.989881,1.000245,0.998989,1.960043,1.976010
Random Forest,"[2, 2, 2, True]",0.977683,0.977514,1.494990,1.495840,2.910114,2.936459
Decision Tree,"[best, 5, 3, 2, friedman_mse]",0.977368,0.977189,1.499124,1.504789,2.930330,2.957356


### Análise dos resultados dos diferentes algoritmos

O gráfico abaixo apresenta o coeficiente de determinação dos modelos com menor RMSE de cada algoritmo testado. Através dele constata-se que todos alcançaram coeficientes de determinação superior a 98%. Indicando que todos os modelos têm boa capacidade de explicar a variação dos preços da WEGE3. 

In [23]:
grf_det_modelos = px.bar(data_frame=df_modelos.round(3)*100,
                         x=df_modelos.index, 
                         y = 'R2_teste',
                         barmode='group',
                         title= 'Coeficiente de Determinação x Modelos',
                         text='R2_teste',
                         width = 700,
                         height= 400,
                         )
grf_det_modelos.update_yaxes(visible=False)
grf_det_modelos.update_traces(textposition="inside")
grf_det_modelos.update_layout(title_x = 0.5,
                              title_y = 0.97, 
                              xaxis_showgrid=False, 
                              xaxis_tickangle=45, 
                              xaxis_title="",
                              margin=dict(l=10, r=10, b=0, 
                                          t=35, pad=0)
                              )
grf_det_modelos

Ao analisar as métricas MAE e RMSE dos modelos, tanto com os dados de treino quanto com os dados de teste, o modelo desenvolvido com o algoritmo XGBoost foi o que apresentou menores erros médios. Por isso foi o escolhido para resolver o problema em questão. 

O gráfico a seguir apresenta as métricas de erro de cada modelo.

In [24]:
grf_erro_modelos = px.bar(data_frame=df_modelos,
                          x=df_modelos.index, 
                          y=df_modelos.drop(['Parametros', 
                                             'R2_treino', 
                                             'R2_teste'
                                             ], axis=1
                                             ).columns,
                          barmode='group',
                          title= 'Métricas de Erro x Modelos',
                          width = 700,
                          height= 400,
                          )
grf_erro_modelos.update_layout(title_x = 0.5,
                               title_y = 0.97, 
                               xaxis_showgrid=False, 
                               xaxis_tickangle=45,
                               xaxis_title="",
                               yaxis_title="",
                               margin=dict(l=10, r=10, b=0, 
                                           t=35, pad=0)
                               )
grf_erro_modelos

### Conclusão e recomendações para a melhoria do modelo

Pôde-se constatar também que o RMSE encontrado no teste do modelo escolhido (1,42), representa apenas 10,35% do desvio padrão dos fechamentos ajustados (13,66). Tal constatação indica que o modelo escolhido pode ser utilizado como ferramenta de apoio a tomada de decisão dos investidores. Sendo assim conclui-se que o projeto atingiu seu objetivo.

In [25]:
df_modelos['RMSE_teste']['XGBoost']/df_original['Adj Close'].std()

0.10326097885464472

No entanto, recomenda-se que sejam buscadas novas métricas que possam melhoras o desempenho do modelo. Recomenda-se, também, que sejam desenvolvidos modelos com base no histórico de cotações de outras ações, para apoiar a tomada de decisão dos investidores de forma mais abrangente.